### Webscraping personal notebook. To scrape tables from website, and create DataFrames.
- See notebook marathons_eda for exploration and nformation for final presentation

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

Create for loop to get a single race   
Write function to create for loop for multiple dfs

In [2]:
# Number of pages of results for each race. 
# From Readme file for class assignment (instructors got this info by using Postman app)
# Need this info to build the loop and function later (in personal work) when build 
# function / for loop.

pgs_full_2016 = 154   
pgs_full_2017 = 147   
pgs_full_2018 = 85   
pgs_full_2019 = 113   
pgs_half_2016 = 898   
pgs_half_2017 = 892   
pgs_half_2018 = 598   
pgs_half_2019 = 690   

In [3]:
urlbase_full_2019 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2019-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_full_2018 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2018-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_full_2017 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2017-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_full_2016 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2016-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_half_2019 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Half-Marathon/2019-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_half_2018 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Half-Marathon/2018-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_half_2017 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Half-Marathon/2017-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_half_2016 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Half-Marathon/2016-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='

### The steps to get a DataFrame from one page of results look like this:
STEP 1: Build a URL by combining the base url with a specific page number  
STEP 2: Use requests.post() to get the results of the post  
STEP 3: Make a soup from results.text  
STEP 4: Look at the soup to identify the table you want based on one of its attributes (like class)   
STEP 5: Pass the table as a string to pandas read_html()   
STEP 6: What does that look like? What is the datatype?  
STEP 7: Keep working with the data until you have it a DataFrame  

In [4]:
# STEP 1: Build a URL by combining the base url with a specific page number

base_f_19 = urlbase_full_2019
page = 99             #To pull just one page to take a look at 
url_f_19 = base_f_19 + str(99)  #This appends page #99 to the end of the URL. 
print(url_f_19)

https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2019-Results?gender=&agegroup=&bib=&firstname=&lastname=&page=99


### Make a request using the `requests` [library](https://requests.readthedocs.io/en/master/user/quickstart/)
- `request.get()` uses http GET to get a webpage
- `request.post()` uses http POST when the webpage is submitting a form
- checking the [`status_code`](https://www.restapitutorial.com/httpstatuscodes.html) on the result let's you know your request was successful

In [5]:
# STEP 2: Use requests.post() to get the results of the post

response_f_19 = requests.post(url_f_19)   #url defined above, it's page 99 (only) of 2019 full marathon.
print(type(response_f_19))            #To see what type "response" is. It's requests.
soup_f_19 = BS(response_f_19.text, 'lxml')   #parses response (page 99 of 2019 full marathon) into a soup
print(type(soup_f_19))  

<class 'requests.models.Response'>
<class 'bs4.BeautifulSoup'>


In [6]:
# STEP 3: Make a soup from results.text   This makes a new TABLE for full 2019 marathons

tables_f_19 = soup_f_19.find_all('table',
                  attrs = {'class': 'table table-responsive table-bordered'})

In [7]:
# STEP 4: Look at the soup to identify the table you want based on one of its attributes (like class)

len(tables_f_19)

3

In [8]:
# STEP 5: Pass the table as a LIST to pandas read_html() 

# We can deduce that the first table is the one we want (position 0)
#This code returns a LIST of the DATAFRAMES.. we're only asking for the one in position [0], 
#  so we'll just get one df.

results_list_f_19 = pd.read_html(str(tables_f_19[0]))   #index position 0

In [9]:
len(results_list_f_19)

1

In [10]:
# STEP 6: What does that look like? What is the datatype?  It's a list

print(type(results_list_f_19)) 

<class 'list'>


In [11]:
# STEP 7: This creates a df from the above list

df_f_19 = results_list_f_19[0]

In [12]:
df_f_19.head()

,Overall,Bib,Name,Time
0,99999,32379,Raquel Flores,00:00:00
1,99999,30292,Kyle Domingos,00:00:00
2,99999,32850,Paul Dillard,00:00:00
3,99999,31415,Nicole Bennett,00:00:00
4,99999,32995,Rudy Novak,00:00:00


## NEXT:  Creating for loop and function to scrape all tables for all races, then saving the resulting dfs as .csv files

1. Create a for loop to pull all results for a single race
    - build a complete url by combining a base url and page number (need to do once for each page, e.g. 113 times for 2019).
    - TIP: As I'm building each line of the for loop, can print each one  to see what it does
    - Use requests.post() to get the results of the post. This is necessary because the first table on the website is a form. The "post" tells it to run and give back page 1, then page 2, etc. 
        - If I wanted either of the otehr two tables of already-populated leaderboards, could use .get instead of .post
    - Then make soup from the post
    - Find the data we want from inside the soup. Pull in all things from the website named 'table' that have attributes 'class' : 'table table-responsive table-bordered'. That info. was gleaned from using Inspect on the website to look at the code.
    - Pass the table as a string to pandas read_html()
    
Function notes from DataCamp:
- When you DEFINE a function, write parameters in function header
- When you CALL a function, you pass arguments into the function
- Docstring: after writing def xxx(xxx), next line '''What the function does''' in plain language, for reference. Like '''Returns the square of a value'''
- Top line, header, that specifies the function name and parameter(s) is the SIGNATURE

In [38]:
# This is a for loop to create a df for only 2016. 
# Preliminary step to creating generic for loop to use inside a function

full_2016_df = pd.DataFrame()  
for page in range(1, pgs_full_2016+1):     #getting pages 1 through end of pages from list of pgs at top
    url = urlbase_full_2016 + str(page)    #page doest't have to be defined
    response = requests.post(url)     #post info and take pg X of results
    soup = BS(response.text, 'lxml')  #translates from http to soup, .text says only take the meat, not the packaging
    tables = soup.find_all('table',
                  attrs = {'class': 'table table-responsive table-bordered'})
    results_list = pd.read_html(str(tables[0]))    #output of read_html is a list
    df = results_list[0]
    full_2016_df = full_2016_df.append(df)         #This takes the results thus far in df and appends next page.

full_2016_df.head()

,Overall,Bib,Name,Time
0,1,1,Scott Wietecha,02:25:42
1,2,3,Brian Shelton,02:34:43
2,3,1047,Christopher Capps,02:38:43
3,4,1052,Jason Grimes,02:45:06
4,5,1046,David Adams,02:47:28


In [14]:
# This creates the function with generic terminology that gets defined in the SIGNATURE (two attributes in the first line)
# Once this is created, it can be "called" in future cells, to run the for loop on a specific year.

def scrape_marathons(page_count, urlbase):      #name is usually a verb
    '''Returns a dataframe with all pages for a single race'''
    
    races_df = pd.DataFrame()  

    for page in range(1, page_count+1):     #getting pages 1 through end of pages, as defined at top of notebook
        url = urlbase + str(page)           #page doest't have to be defined
        response = requests.post(url)       #post info and take pg X of results
        soup = BS(response.text, 'lxml')    #translates from http to soup, .text says only take the meat, not the packaging
        tables = soup.find_all('table',
                      attrs = {'class': 'table table-responsive table-bordered'})
        results_list = pd.read_html(str(tables[0]))    #output of read_html is a list
        df = results_list[0]
        races_df = races_df.append(df)             #every time thru appends 25 rows.
    return races_df

In [15]:
# Already created 2016 df two cells above. No need to run the function on that.

full_2016_df.shape

(3842, 4)

In [43]:
# First use of "scrape_marathons" FUNCTION created above, 
# to pull all data for 8 DFs (4 half and 4 full marathons)

full_2017_df = scrape_marathons(pgs_full_2017, urlbase_full_2017)
print(full_2017_df.shape)
print(full_2017_df.head())

(3651, 4)
   Overall   Bib            Name      Time
0        1     1  Scott Wietecha  02:40:25
1        2  1145    Ryan Regnier  02:56:28
2        3  1147  Daniel Everett  03:00:55
3        4  1029   Khris Vickroy  03:04:47
4        5  1119  Marcus Dilallo  03:04:53


In [41]:
full_2018_df = scrape_marathons(pgs_full_2018, urlbase_full_2018)
full_2018_df.shape

(2115, 4)

In [18]:
full_2019_df = scrape_marathons(pgs_full_2019, urlbase_full_2019)
full_2019_df.shape

(2811, 4)

In [37]:
half_2016_df = scrape_marathons(pgs_half_2016, urlbase_half_2016)
half_2016_df.shape

(22429, 4)

In [21]:
half_2017_df = scrape_marathons(pgs_half_2017, urlbase_half_2017)
half_2017_df.shape

(22292, 4)

In [22]:
half_2018_df = scrape_marathons(pgs_half_2018, urlbase_half_2018)
half_2018_df.shape

(14940, 4)

In [23]:
half_2019_df = scrape_marathons(pgs_half_2019, urlbase_half_2019)
half_2019_df.shape

(17236, 4)

The next series of cells is to convert these dfs into .csv files
These will be read into a new notebook for exploration, named marathons_eda

In [39]:
full_2016_df = full_2016_df.to_csv('full_2016.csv', index = False)

In [28]:
full_2017_df = full_2017_df.to_csv('full_2017.csv', index = False)

In [42]:
full_2018_df = full_2018_df.to_csv('full_2018.csv', index = False)

In [32]:
full_2019_df = full_2019_df.to_csv('full_2019.csv', index = False)

In [33]:
half_2016_df = half_2016_df.to_csv('half_2016.csv', index = False)

In [34]:
half_2017_df = half_2017_df.to_csv('half_2017.csv', index = False)

In [35]:
half_2018_df = half_2018_df.to_csv('half_2018.csv', index = False)

In [36]:
half_2019_df = half_2019_df.to_csv('half_2019.csv', index = False)

### Success! End of this notebook
### EDA is being done in notebook named marathons_eda